# Intro to Data Analysis with Pandas & Jupyter, November 2019

Workshop lead: Sam Bail [@spbail](http://twitter.com/spbail)

# Overview
The goal of this workshop is to give learners an intro to data analysis with Python using Pandas and Jupyter. 
We will first go through the process of loading data from CSV files, inspecting and cleaning the data. As a second step, we will analyse the data and draw some insights about cancer treatment from it. 

The tutorial is structured as follows:

- Intro and background
- Part 0: Quick Jupyter exercise
- Part 1: Loading and inspecting data
- Part 2: Data cleaning
- Part 3: Data analysis
- Part 4: Summary

**Note that this tutorial is only intended as an introduction to some basic concepts of Pandas. It is in no means intended to be comprehensive, and there are a lot of useful functions a beginner needs to know to do in-depth data analysis. We hope that this tutorial sets you up for self-guided learning to master the full range of necessary Pandas tools.**

## How to follow along with the workshop
- You can run every cell in the notebook as we go along using the shortcut Shift+Enter
- You will encounter a few <span style="color:blue">*** DIY exercise ***</span> blocks where you'll get a few minutes to try out what you've just learned
- Feel free to save and download your notebook from Binder at the end since Binder deletes notebooks after 12 hours.

# Intro

## What is Jupyter (and the Jupyter ecosystem...)?
- IPython is an interactive Python shell (just type "ipython" to start it)
- Jupyter is a Python library that provides a web-based UI on top of ipython to create notebooks with code and output
- JupyterLab provides some additional features on top of Jupyter, e.g. a file browser
- Binder is a web-based hub for containers that contain your Python environment and renders notebooks based on a git repo

## What is Pandas/Matplotlib/Pyplot/Seaborn?

- Pandas is a Python library for data manipulation and analysis. It offers data structures and operations for manipulating numerical tables and time series.
- Matplotlib is a Python 2D plotting library. Pyplot is a collection of command style functions in matplotlib that make matplotlib work like MATLAB. While we mostly use Seaborn, we sometimes fall back to using Pyplot functions for certain aspects of plotting.
- Seaborn is a Python data visualization library based on matplotlib. It's kind of like a nicer version of Pyplot.


# Part 0: Quick Jupyter exercise (10 mins)

Notebooks are basically just interactive ipython terminals, intermixed with markdown text:
- Cells can be either code or markdown
- You can execute any kind of Python code
- Variables persist between cells
- The notebook doesn't care about the order of cells, just the order of executing it in order to remember variables. However, "run all" executes your cells top to bottom.

Notebooks have **two modes**: a) editing the cells and b) navigating the notebook (command mode).
- You can navigate around the notebook in command mode by clicking cells or using the arrow keys
- Depending on the environment you're using (Jupyter notebook, Jupyter lab, Google Colab...) there will be a different visual cue to indicate the mode a cell is in
- In order to edit a cell, you can hit enter or double-click it.
- To execute the cell content, hit Shift+Enter to run the cell
- To get out of edit mode, hit the Escape key

Some additional helpful shortcuts:
- The default type for a cell is code. In command mode, hit *m* to make a cell markdown and *y* to make it code
- Hit *a* in command mode to create a new cell *above* the current one
- Hit *b* in command mode to create a new cell *below* the current one
- *Tab* autocompletes methods (like in ipython)
- *Shift+Tab* shows you the docstring for the outer function of the line your cursor is in
- Hit *dd* in command mode to delete a cell. 
- *Cmd+z* undoes operations in the highlighted cell, *z* undoes cell operations in the notebook (e.g. deleting a cell)

In [ ]:
# Examples

print('Hello world!')

import math
math.ceil(4.3)

## <span style="color:blue">*** DIY exercise ***</span>
Try out using only your keyboard and shortcuts for these two tasks:
- Create a new *markdown* cell below this one, write a few lines and format them to look like a header and bullets.
- Create a new *code* cell above the first one one, import your favorite Python function, check out the docstring, and execute the code (e.g. `os.getcwd()`, `random.random()`, etc.)

# Part 1: Loading and inspecting the data (20 mins)

Before we can start answering questions about the data we need to do a little bit of exploratory analysis.The first thing we need to do when working with a new dataset is to get an idea of what the data looks like. We start by loading the data into memory. Pandas comes with a built-in `read_csv` function that we can use to read CSV files and load them directly to a pandas `DataFrame` object. 

In [ ]:
# We need to import the libraries to start with
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# This command makes charts show inline in a notebook
%matplotlib inline

# Making the figures show up a little larger than default size
plt.rcParams['figure.figsize'] = [10,6]

There are a lot of different ways to read data into a dataframe - from lists, dicts, CSVs, databases... Here's an example of querying a database table to pull data into a dataframe:

In [ ]:
import psycopg2

# Set up a connection to a public SQL database
conn = psycopg2.connect("""
    host='hh-pgsql-public.ebi.ac.uk' dbname='pfmegrnargs' user='reader' password='NWDMCE5xdipIjRrp'
""")

# Write a SQL query to pull the data
query = "SELECT * FROM rnacen.protein_info LIMIT 5"

# Execute the query and store the result in a dataframe
df_from_db = pd.read_sql(query, conn)

# Show the resulting dataframe
df_from_db

## What is a DataFrame?
A DataFrame is a 2-dimensional labeled data structure with columns of potentially different types. You can think of it like a spreadsheet or SQL table, or a dict of Series objects. It is generally the most commonly used Pandas object. 

Pandas borrows the concept of DataFrame from the statistical programming language R.

Let's take a look at the data to familiarize ourselves with the format and data types.

In [ ]:
# Read data from a CSV into a dataframe
# This is the data we're going to be working with!
tx = pd.read_csv('./mock_treatment_starts_2016.csv')

In [ ]:
# We already did this: Just using the name of the dataframe will print the entire output
# If there are too many rows, Jupyter will print the top few and bottom few rows
# with a "..." to indicate that there are more rows
tx

In [ ]:
# The head(n) function shows the first n rows in a dataframe.
# If no n is specified, it defaults to 5 rows.
tx.head()

In [ ]:
# You can also use the sample() function to get n random rows in the dataframe
# NOTE: sample() only works in newer versions of pandas (0.16.1 and upwards)
tx.sample(5)

In [ ]:
# Then len function gives us the number of rows in the dataframe
len(tx)

In [ ]:
# The dtypes property of a dataframe shows the datatypes of every column in a dataframe.
tx.dtypes

In [ ]:
# The columns attribute of a dataframe contains the column names
# We'll talk about the "Index" later!
tx.columns

In [ ]:
# The describe function shows some basic statistics for numeric columns
# We only have one here, so this isn't very interesting
tx.describe()

## <span style="color:blue">*** DIY exercise ***</span>
Create a new cell below and print the first ten rows of the "tx" dataframe.

## Accessing columns in a dataframe

<span style="color:blue">**Note: We will be applying `head()` to some results in this tutorial to keep the output short. When working with a real dataset, keep in mind that you might be hiding some relevant records if you always use `head()` or `sample()`!**</span>

In Pandas, you can access a specific column using the following notation which returns a **Series** (not a dataframe) - a Series is simply a vector, aka a 1-dimensional data structure similar to a list.

In [ ]:
# Return the PatientID column as a Series
tx['PatientID'].head()

In [ ]:
# Check the type to show that this indeed returns a Series object
type(tx['PatientID'])

In [ ]:
# The alternative notation for accessing a column in a dataframe
# Some people prefer the . notation, others the [] notation.
# Personally, I prefer using [] for visibility and consistency
tx.PatientID.head()

In [ ]:
# And this is how you access two columns of a dataframe.
# Note that this will return a dataframe again, not a series (because a series has only one column...)
# Also note the double square brackets - you're passing a *list* of columns as an argument
tx[['PatientID', 'Dosage']].head()

In [ ]:
# Check the type to confirm that this returns a DataFrame type
type(tx[['PatientID', 'TreatmentStart']])

In [ ]:
# This way we can now do some more data exploration, e.g. looking at unique patient IDs
sorted(tx['PatientID'].unique())

## <span style="color:blue">*** DIY exercise ***</span>
Create a new cell below and print the list of unique drugs in the dataframe.

## Accessing rows in a dataframe
In addition to slicing by column, we often want to get the record where a column has a specific value, e.g. a specific Patient_ID here. This can be done using the following syntax:

In [ ]:
# Access the record(s) for a specific patient ID
tx.loc[tx['PatientID'] == 'PT20']

In [ ]:
# This is equivalent with the following shorter notation
# I prefer to always use loc to be more explicit
tx[tx['PatientID'] == 'PT20']

In [ ]:
# You can also use boolean conditions in the selector
tx.loc[(tx['PatientID'] == 'PT20') & (tx['Drug'] == 'Cisplatin')]

## <span style="color:blue">*** DIY exercise ***</span>
Create a new cell below and show all rows where the drug dosage for Cisplatin is less than 180.

## Sorting dataframes
Sorting a dataframe by one or multiple columns is super easy:

In [ ]:
# Sort by earliest treatment start date, i.e. in ascending order
tx.sort_values('TreatmentStart').head()

# NOTE: sort_values only works in Pandas 0.17.0 and up. This is an older version:
# tx.sort('TreatmentStart').head()

In [ ]:
# Sort by latest treatment start, i.e. in descending order
tx.sort_values('TreatmentStart', ascending=False).head()

# NOTE: sort_values only works in Pandas 0.17.0 and up. This is an older version:
# tx.sort('TreatmentStart', ascending=False).head()

In [ ]:
# Finally, you can also sort by a list of column. If you want to change the 
# ascending/descending orders, pass a list of booleans to the `ascending` parameter!
tx.sort_values(['PatientID', 'TreatmentStart']).head()

# NOTE: sort_values only works in Pandas 0.17.0 and up. This is an older version:
# tx.sort(['PatientID', 'TreatmentStart']).head()

**Note: Any operations on a dataframe are *not* permanent, i.e. they only modify the current output, but not the actual dataframe. If you want to preserve the sorting, for example, you have to either assign the output to a new variable, or use the `inplace=True` argument. This will not create any output but actually modify the dataframe.**

In [ ]:
# Show the dataframe
tx.head()

In [ ]:
# Use the inplace keyword to modify the dataframe
# Note that you can also sort by a list of columns
tx.sort_values(['PatientID', 'TreatmentStart'], inplace=True)

In [ ]:
# Check out the permanently sorted dataframe
tx.head()

## <span style="color:blue">*** DIY exercise ***</span>
Create a new cell below and sort the dataframe by drug (ascending, i.e. alphabetically) and then dosage (descending order, i.e. highest dosage first).

# Part 2: Data cleaning (15 mins)

In [ ]:
# Remember the dtypes property... the TreatmentStart column should really be a date, right?
tx.dtypes

## Date conversion

Right away we can see that the date field TreatmentDate is stored as string (object). It might be useful to convert it to **Datetime** objects so that we can perform common date arithmetic on them, like checking if a date came before or after another date, or calculating the number of days between two dates.

In [ ]:
# This assigns the datetime version of the TreatmentStart column to a column with the same name
tx['TreatmentStart'] = pd.to_datetime(tx['TreatmentStart'])

In [ ]:
# Check the types now - we have a datetime64 type!
tx.dtypes

In [ ]:
# This is the alternative notation to access a column in a dataframe
tx.TreatmentStart = pd.to_datetime(tx.TreatmentStart)

## Sidebar: Copying dataframes
Note that if you assign a dataframe to a new variable, it will reference the same underlying object as the original dataframe. This means that any modification you make to the new dataframe will also be applied to the old one. Use the `copy()` function to make a new copy of the dataframe by value.

In [ ]:
tx2 = tx

# Create a new dummy column in the "copy" of our dataframe
tx2['NewColumn'] = 1

In [ ]:
# tx2 now has the new column...
tx2.head()

In [ ]:
# ... but so does our original dataframe. This is not really what we want!
tx.head()

In [ ]:
# Drop the dummy column from the original dataframe
# axis=1 means we're dropping columns, and we need to use inplace=True to make it permanent!
if 'NewColumn' in tx.columns:
    tx.drop('NewColumn', axis=1, inplace=True)

# Then make a real copy of the "clean" tx dataframe
tx2 = tx.copy()

# Add the dummy column to tx2 and confirm that the original tx doesn't have it
tx2['NewColumn'] = 1
tx.head()

## <span style="color:blue">*** DIY exercise ***</span>
Create a copy of the tx dataframe and add a new column TreatmentStartDT that contains the treatment starts as datetime types.

# Part 3: Data analysis (30 minutes)
Let's assume we've loaded the treatment related data from a cancer clinic in order to provide them with some analytical insights around the types of drugs they use on their patient population.

## Question 1: Patients treated at the practice

**How many patients does the practice treat?**

In [ ]:
# Our data frame contains patient IDs and treatment starts -
# let's check if some patients have multiple treatment starts?
# The unique() function returns the number of unique values in a dataframe column.
print('Number of treatment start records:', len(tx))
print('Number of unique patients who start treatment:', len(tx.PatientID.unique()))

So there are 20 unique patients but we have 23 treatment starts, meaning some patients start different treatments in the time that we have data for. This means that if we want to answer the question correctly, we need to make sure to only count unique patients. Let's learn some counting techniques first before coming back to the duplicate issue!

## Question 2: Drugs used at the practice
**What are the drugs used at the practice and how many patients receive those drugs?**

In [ ]:
tx.groupby('Drug')

In [ ]:
# The groupby function works like a groupby in SQL, i.e. it groups the dataframe by the specified
# column and then lets you apply aggregate functions on the grouped values, e.g. counts, sums, means...
# The count function counts the number of rows with non-null values in a column
tx.groupby('Drug').count()

In [ ]:
# Since we are only interested in the number of patients, 
# we select only the relevant column from the resulting dataframe
# Note that "PatientID" might not be the best name for this column
# - we can use a rename() function in Pandas to rename it to something like
# "PatientCount" (skipping the rename step in this tutorial, but feel free to look it up!)
tx.groupby('Drug').count()[['PatientID']]

In [ ]:
# We can also use nunique() which counts the number of *unique* non-null values for each column
# Notice how the numbers are different from the count() result
tx.groupby('Drug').nunique()

## <span style="color:blue">*** DIY exercise ***</span>
With the techniques you just learned in Question 2, think back to Question 1. Create a new cell below and count how many records each patient has in order to spot those patients that receive more than one treatment.

**Additional point to think about:** Depending on what question we want to answer, counting the number of records might not give us the correct answer. Can you think of different questions a clinic might ask to explain why patients have multiple records? What stands out when using nunique() instead of count()?

### A little bit about indexes in dataframes
Notice that in the above example, the "Drug" column is printed in bold. That's because grouping by it has turned it into the **index** of the resulting dataframe.

The index in a dataframe is the "row identifier" - it is generally printed as the column on the left. For example, when we first loaded our data, the index didn't have a name and was just an incrementing integer (scroll up to check!). When you create a groupby object, the index of a resulting dataframe will be the column you group by - in this case, the Drug column became the index.

We frequently **reset** the index in a dataframe for various reasons - in this case, because the index contains data that you want to treat as a column, e.g. for plotting.

In [ ]:
# This is the same groupby we did above. Notice how the "Drug" column is bold
# - it became the index after grouping by it
tx.groupby('Drug').count()[['PatientID']]

In [ ]:
# Reset the index in the grouped dataframe to see what happens:
tx.groupby('Drug').count()[['PatientID']].reset_index()

In [ ]:
# Remember that any operations on the dataframe only modify the output? 
# We didn't *really* group the dataframe or reset the index. 
# The tx dataframe is still the same it was at the beginning.
# We could use inplace=True to make the change permanent.
tx.head()

### Let's plot this! (aka our first Seaborn plot)

In [ ]:
# Let's do the same groupby as above to get the number of patient starts per drug.
# This time, we actually assign the output to a new dataframe `counts` to make the change permanent.
counts = tx\
    .groupby('Drug')\
    .count()[['PatientID']]\
    .reset_index()

# Let's use a simple bar chart in Seaborn to compare counts for the two drugs
# There are several different ways to do the plotting - this is my preferred style,
# but you might prefer different syntax
fig = sns.barplot(data=counts, x='Drug', y='PatientID')
plt.title('Number of patient starts by drug')
plt.ylabel('Number of patient starts')
plt.xlabel('Drug')
plt.show(fig)

## Question 3: Changes to treatment over time
**Do we see any changes in treatment patterns over time?**

Our data shows treatment starts by date. Let's group these starts by month to see if there are any changes of how many patients start on a given drug over time, e.g. because a new drug got approved.

*Note that the data we're using here is dummy data and pretty artificial - oncology clinics see a much higher volume of patients, and drug uptake is usually slower than shown here.*

In [ ]:
# Let's add a new column that only has the treatment month to simplify things
# There are many different ways to do this, we picked a simple one
tx['TreatmentStartMonth'] = tx['TreatmentStart'].astype('datetime64[M]')

# NOTE .astype('datetime64[M]') only works in more recent versions of Pandas, this is an older version:
# from datetime import datetime
# tx['TreatmentStartMonth'] = tx['TreatmentStart'].apply(lambda x: x.replace(day=1))

tx.head()

In [ ]:
# Let's count the number of starts per month per drug to plot it later
# We only want the number of patients, so we filter for that column at the end
drugs_by_month = tx.groupby(['TreatmentStartMonth', 'Drug']).count()[['PatientID']]
drugs_by_month

In [ ]:
# The data already looks interesting... let's plot this 
# Remember to reset_index so we can plot the regular columns
# The "hue" keyword is generally used to distinguish two different categorical variables
# in plots, e.g. in this case the two different drugs
# NOTE: lineplot() only exists in Seaborn version 0.9 and up
fig = sns.lineplot(data=drugs_by_month.reset_index(), 
                   x='TreatmentStartMonth', 
                   y='PatientID',
                   hue='Drug')
plt.title('Number of patient starts by drug and by month')
plt.ylabel('Number of patient starts')
plt.xlabel('Drug')
plt.show(fig)

## <span style="color:blue">*** DIY exercise ***</span>
Plot drugs_by_month as a clustered barplot instead of lineplot. Make sure you're clear about what your x, y, and hue are in this case!

Note that the date labeling on the x axis doesn't look good because Seaborn converts the month back to a datetime. There are several ways to deal with this - can you think of one possible solution that works in this particular case?

## [Optional] Question 4: Dosage and outliers
**Question: What is the average dosage of each drug? Are there any outliers?**

In [ ]:
# An easy first step is to group by the respective drug and use describe()
tx.groupby(['Drug']).describe()

In [ ]:
# This is an example of a more complex way to get aggregates in Pandas
# The agg function takes a dictionary of column:function pairs,
# where "function" can be a built-in function like count, mean, min, etc, 
# or a custom function like a lambda.
tx.groupby(['Drug']).agg({'Dosage': 'mean'})

In [ ]:
# We can also pass a list of functions to a column to get multiple outputs!
tx.groupby(['Drug']).agg({'Dosage': ['count', 'mean', 'std', 'min', 'max']})

In [ ]:
# We can plot this easily in Seaborn - but the outlier squashes our display
fig = sns.boxplot(data=tx, x='Drug', y='Dosage')
plt.show(fig)

In [ ]:
# Use showfliers=False in a boxplot to suppress outliers
fig = sns.boxplot(data=tx, x='Drug', y='Dosage', showfliers=False)
plt.show(fig)

# Part 4: Summary!

We hope this workshop was useful for you. We've only touched on some of the basic concepts of pandas, but we believe this will give you the foundations to keep exploring the data! We covered:

- Basic operations in Jupyter and data loading with Pandas
- Dataframes and Series in pandas
- Basic data inspection (head, describe, dtypes, accessing columns and rows, sorting)
- Grouping and aggregating (count, nunique)
- Indexing in dataframes and reset_index
- Plotting (bar plots, line plots)

**What we didn't learn:**

This is my (biased) list of very frequent Pandas operations that we didn't cover but you'll likely need for data analysis:
- Joining/merging multiple dataframes
- Filtering and de-duplicating dataframes
- More complex modifications of column values, e.g. filling null values, using lambda functions
- More complex aggregates on grouped dataframes (sum, mean, etc)
- Renaming columns (e.g. renaming an aggregate "PatientID" column to something more meaningful like "PatientCount")

Let me know what you think! samanthapbail@gmail.com / [@spbail](http://twitter.com/spbail)